In [3]:
import pandas as pd

# 加载数据集
submit_records = pd.read_csv('./数据/Data_SubmitRecord/SubmitRecord-Class1.csv')
question_attributes = pd.read_csv('./数据/题目属性.csv')
student_attributes = pd.read_csv('./数据/学生属性.csv')

# 检查空值并删除或填充
submit_records = submit_records.dropna()
question_attributes = question_attributes.dropna()
student_attributes = student_attributes.dropna()

# 合并数据集
# 1. 先将题目属性合并到答题记录数据
merged_data = pd.merge(submit_records, question_attributes, on='title_ID', how='left')

# 2. 再将学生属性数据合并到已合并的表格
merged_data = pd.merge(merged_data, student_attributes, on='student_ID', how='left')

# 检查合并后的数据集
print(merged_data.head())


   index_x   class          time               state  score_x  \
0        0  Class1  1.704210e+09  Absolutely_Correct        3   
1        1  Class1  1.704210e+09  Absolutely_Correct        3   
2        2  Class1  1.704210e+09  Absolutely_Correct        3   
3        3  Class1  1.704209e+09  Absolutely_Correct        3   
4        4  Class1  1.704208e+09  Absolutely_Correct        4   

                        title_ID                       method  memory  \
0  Question_bumGRTJ0c8p4v5D6eHZa  Method_Cj9Ya2R7fZd6xs1q5mNQ     320   
1  Question_62XbhBvJ8NUSnApgDL94  Method_gj1NLb4Jn7URf9K2kQPd     356   
2  Question_ZTbD7mxr2OUp8Fz6iNjy  Method_5Q4KoXthUuYz3bvrTDFm     196   
3  Question_xqlJkmRaP0otZcX4fK3W  Method_m8vwGkEZc3TSW2xqYUoR     308   
4  Question_FNg8X9v5zcbB1tQrxHR3  Method_Cj9Ya2R7fZd6xs1q5mNQ     320   

  timeconsume            student_ID  index_y  score_y knowledge  \
0           3  8b6d1125760bd3939b6e       42        3     b3C9s   
1           3  8b6d1125760bd3939b6e 

In [4]:
# 按知识点分组并计算平均得分
knowledge_mastery = merged_data.groupby('knowledge')['score_x'].mean().reset_index()
knowledge_mastery.columns = ['Knowledge Point', 'Average Score']

# 输出结果检查
print(knowledge_mastery)


  Knowledge Point  Average Score
0           b3C9s       1.036002
1           g7R2j       0.950785
2           k4W1c       1.042301
3           m3D1v       0.964292
4           r8S3g       0.176358
5           s8Y2f       0.862955
6           t5V9e       0.599556
7           y9W5d       1.041667


In [5]:
from pyecharts.charts import Bar
from pyecharts import options as opts

# 提取知识点和平均得分数据
knowledge_points = knowledge_mastery['Knowledge Point'].tolist()
average_scores = knowledge_mastery['Average Score'].tolist()

# 创建柱状图
bar = (
    Bar()
    .add_xaxis(knowledge_points)
    .add_yaxis("Average Score", average_scores)
    .set_global_opts(
        title_opts=opts.TitleOpts(title="Knowledge Point Mastery Levels"),
        xaxis_opts=opts.AxisOpts(name="Knowledge Points"),
        yaxis_opts=opts.AxisOpts(name="Average Score"),
    )
)

# 渲染图表到 HTML 文件
bar.render("knowledge_mastery_bar_chart.html")


'D:\\Python\\数据可视化\\实验六\\knowledge_mastery_bar_chart.html'

In [8]:
import datetime

# 转换时间戳
merged_data['time'] = pd.to_datetime(merged_data['time'], unit='s')
merged_data['hour'] = merged_data['time'].dt.hour

# 统计每个小时的答题次数
hourly_distribution = merged_data.groupby('hour').size().reset_index(name='Answer Count')

print(hourly_distribution)


    hour  Answer Count
0      0           255
1      1           554
2      2          1321
3      3          1122
4      4           800
5      5          1057
6      6          1746
7      7          1084
8      8          1482
9      9          1140
10    10          1355
11    11          2133
12    12          2005
13    13          2136
14    14          2525
15    15          1493
16    16           379
17    17           143
18    18            17
19    19             9
20    20             5
21    21             9
22    22            54
23    23           136


In [10]:
from pyecharts.charts import Bar
from pyecharts import options as opts

# 创建柱状图
bar = (
    Bar()
    .add_xaxis([f"{i}点" for i in hourly_distribution['hour']])
    .add_yaxis("答题次数", hourly_distribution['Answer Count'].tolist())
    .set_global_opts(
        title_opts=opts.TitleOpts(title="答题高峰时段分布"),
        xaxis_opts=opts.AxisOpts(name="时段"),
        yaxis_opts=opts.AxisOpts(name="答题次数"),
    )
)

# 渲染图表
bar.render("hourly_distribution_bar_chart.html")


'D:\\Python\\数据可视化\\实验六\\hourly_distribution_bar_chart.html'

In [11]:
from pyecharts.charts import HeatMap

# 创建热力图数据，将每个小时的答题次数作为不同学习行为的分布
heatmap_data = [[hour, 0, count] for hour, count in zip(hourly_distribution['hour'], hourly_distribution['Answer Count'])]

# 绘制热力图
heatmap = (
    HeatMap()
    .add_xaxis([f"{i}点" for i in range(24)])
    .add_yaxis("学习行为", ["答题次数"], heatmap_data)
    .set_global_opts(
        title_opts=opts.TitleOpts(title="答题高峰时段分布"),
        visualmap_opts=opts.VisualMapOpts(max_=max(hourly_distribution['Answer Count'])),
    )
)

# 渲染图表到 HTML 文件
heatmap.render("hourly_distribution_heatmap_corrected.html")


'D:\\Python\\数据可视化\\实验六\\hourly_distribution_heatmap_corrected.html'

In [12]:
# 示例分析：以记忆耗费量为学习模式之一，计算不同记忆耗费下的平均得分
memory_mastery = merged_data.groupby('memory')['score_x'].mean().reset_index()
memory_mastery.columns = ['Memory Consumed', 'Average Score']

print(memory_mastery)


     Memory Consumed  Average Score
0                  0       0.000000
1                160       0.000000
2                164       0.000000
3                168       0.636364
4                172       0.344828
..               ...            ...
140             8668       0.000000
141             8800       0.000000
142             9536       0.000000
143             9564       0.000000
144             9664       0.000000

[145 rows x 2 columns]


In [13]:
from pyecharts.charts import Boxplot

# 创建箱线图
boxplot = Boxplot()
x_data = memory_mastery['Memory Consumed'].tolist()
y_data = [memory_mastery['Average Score'].tolist()]

boxplot.add_xaxis(x_data)
boxplot.add_yaxis("Knowledge Mastery", boxplot.prepare_data(y_data))

# 设置全局选项
boxplot.set_global_opts(
    title_opts=opts.TitleOpts(title="学习模式与知识掌握关系"),
    xaxis_opts=opts.AxisOpts(name="Memory Consumed"),
    yaxis_opts=opts.AxisOpts(name="Average Score"),
)

# 渲染图表
boxplot.render("memory_mastery_boxplot.html")


'D:\\Python\\数据可视化\\实验六\\memory_mastery_boxplot.html'

In [14]:
# 计算题目难度和正确率
difficulty_analysis = merged_data.groupby('title_ID').apply(
    lambda x: pd.Series({
        'Average Score': x['score_x'].mean(),
        'Correct Rate': (x['state'] == 'Absolutely_Correct').mean()
    })
).reset_index()

print(difficulty_analysis)


                         title_ID  Average Score  Correct Rate
0   Question_3MwAFlmNO8EKrpY5zjUd       0.655405      0.117761
1   Question_3oPyUzDmQtcMfLpGZ0jW       0.366961      0.139975
2   Question_4nHcauCQ0Y6Pm8DgKlLo       1.091451      0.206759
3   Question_5fgqjSBwTPG7KUV3it6O       1.083643      0.117100
4   Question_62XbhBvJ8NUSnApgDL94       0.963265      0.204082
5   Question_6RQj2gF3OeK5AmDvThUV       1.094675      0.328402
6   Question_7NJzCXUPcvQF4Mkfh9Wr       0.968586      0.282723
7   Question_Az73sM0rHfWVKuc4X2kL       1.085766      0.233577
8   Question_BW0ItEaymH3TkD6S15JF       0.221207      0.221207
9   Question_EhVPdmlB31M8WKGqL0wc       1.192744      0.267574
10  Question_Ej5mBw9rsOUKkFycGvz2       0.943888      0.240481
11  Question_FNg8X9v5zcbB1tQrxHR3       1.019827      0.163569
12  Question_Jr4Wz5jLqmN01KUwHa7g       0.983640      0.269939
13  Question_Mh4CZIsrEfxkP1wXtOYV       0.782278      0.260759
14  Question_NixCn84GdK2tySa5rB1V       1.116279      0

C:\Windows\Temp\ipykernel_5872\4129849964.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  difficulty_analysis = merged_data.groupby('title_ID').apply(


In [15]:
from pyecharts.charts import Scatter

# 创建散点图
scatter = (
    Scatter()
    .add_xaxis(difficulty_analysis['Average Score'].tolist())
    .add_yaxis("Correct Rate", difficulty_analysis['Correct Rate'].tolist())
    .set_global_opts(
        title_opts=opts.TitleOpts(title="题目难度与正确率"),
        xaxis_opts=opts.AxisOpts(name="Average Score"),
        yaxis_opts=opts.AxisOpts(name="Correct Rate"),
    )
)

# 渲染散点图到 HTML
scatter.render("difficulty_analysis_scatter.html")


'D:\\Python\\数据可视化\\实验六\\difficulty_analysis_scatter.html'

In [16]:
from pyecharts.charts import Scatter
from pyecharts import options as opts

# 构造数据：假设 `difficulty_analysis` 包含 `Average Score`（难度）和 `Correct Rate`（正确率）
x_data = difficulty_analysis['Average Score'].tolist()
y_data = difficulty_analysis['Correct Rate'].tolist()

scatter = (
    Scatter()
    .add_xaxis(x_data)
    .add_yaxis("题目正确率", y_data)
    .set_global_opts(
        title_opts=opts.TitleOpts(title="题目难度与正确率关系"),
        xaxis_opts=opts.AxisOpts(name="题目难度"),
        yaxis_opts=opts.AxisOpts(name="正确率"),
        visualmap_opts=opts.VisualMapOpts(type_="color", max_=max(y_data)),
    )
)

# 渲染图表
scatter.render("difficulty_correct_rate_scatter.html")


'D:\\Python\\数据可视化\\实验六\\difficulty_correct_rate_scatter.html'

In [18]:
from pyecharts.charts import Boxplot

# 假设根据难度对题目分成了几个组，并计算每组的正确率分布
difficulty_groups = difficulty_analysis.groupby(pd.cut(difficulty_analysis['Average Score'], bins=5))['Correct Rate']
box_data = [list(group) for _, group in difficulty_groups]

boxplot = Boxplot()
boxplot.add_xaxis(["难度组1", "难度组2", "难度组3", "难度组4", "难度组5"])
boxplot.add_yaxis("正确率分布", boxplot.prepare_data(box_data))

# 设置全局选项
boxplot.set_global_opts(
    title_opts=opts.TitleOpts(title="不同难度题目的正确率分布"),
    xaxis_opts=opts.AxisOpts(name="题目难度分组"),
    yaxis_opts=opts.AxisOpts(name="正确率"),
)

# 渲染图表
boxplot.render("difficulty_correct_rate_boxplot.html")


C:\Windows\Temp\ipykernel_5872\2693389299.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  difficulty_groups = difficulty_analysis.groupby(pd.cut(difficulty_analysis['Average Score'], bins=5))['Correct Rate']


'D:\\Python\\数据可视化\\实验六\\difficulty_correct_rate_boxplot.html'

In [21]:
# 分组计算题目难度、正确率和答题次数，避免警告
difficulty_analysis = merged_data.groupby('title_ID', as_index=False).apply(
    lambda x: pd.Series({
        'Average Score': x['score_x'].mean(),
        'Correct Rate': (x['state'] == 'Absolutely_Correct').mean(),
        'Answer Count': x.shape[0]  # 答题次数
    })
).reset_index(drop=True)

print(difficulty_analysis.head())  # 检查数据结构


                        title_ID  Average Score  Correct Rate  Answer Count
0  Question_3MwAFlmNO8EKrpY5zjUd       0.655405      0.117761        1036.0
1  Question_3oPyUzDmQtcMfLpGZ0jW       0.366961      0.139975         793.0
2  Question_4nHcauCQ0Y6Pm8DgKlLo       1.091451      0.206759         503.0
3  Question_5fgqjSBwTPG7KUV3it6O       1.083643      0.117100        1076.0
4  Question_62XbhBvJ8NUSnApgDL94       0.963265      0.204082         490.0


C:\Windows\Temp\ipykernel_5872\720338477.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  difficulty_analysis = merged_data.groupby('title_ID', as_index=False).apply(


In [28]:
from pyecharts.charts import Scatter
from pyecharts import options as opts

# 提取数据
x_data = difficulty_analysis['Average Score'].tolist()
y_data = difficulty_analysis['Correct Rate'].tolist()
bubble_sizes = difficulty_analysis['Answer Count'].tolist()

# 设置横轴范围和刻度间隔
min_difficulty = min(x_data) - 0.1  # 设置略低于最小值的范围
max_difficulty = max(x_data) + 0.1  # 设置略高于最大值的范围
interval = 0.1  # 设定合理的刻度间隔

# 创建气泡图
bubble_chart = (
    Scatter()
    .add_xaxis(x_data)
    .add_yaxis(
        "题目正确率",
        [list(z) for z in zip(y_data, bubble_sizes)],  # 包括大小信息
        symbol_size=bubble_sizes  # 使用答题次数作为气泡大小
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="题目难度与正确率关系（气泡图）"),
        xaxis_opts=opts.AxisOpts(
            name="题目难度",
            min_=min_difficulty,
            max_=max_difficulty,
            interval=interval
        ),
        yaxis_opts=opts.AxisOpts(name="正确率"),
        visualmap_opts=opts.VisualMapOpts(type_="size", max_=max(bubble_sizes))
    )
)

# 渲染图表
bubble_chart.render("difficulty_correct_rate_bubble.html")

'D:\\Python\\数据可视化\\实验六\\difficulty_correct_rate_bubble_with_title.html'

In [50]:
from pyecharts.charts import Boxplot
from pyecharts import options as opts
import pandas as pd

# 计算题目难度、正确率和答题次数
difficulty_analysis = merged_data.groupby('title_ID').apply(
    lambda x: pd.Series({
        'Average Score': x['score_x'].mean(),
        'Correct Rate': (x['state'] == 'Absolutely_Correct').mean(),
        'Answer Count': x.shape[0]
    })
).reset_index()

# 将难度分段
difficulty_analysis['Difficulty Group'] = pd.cut(difficulty_analysis['Average Score'], bins=5)

# 准备箱线图数据
box_data = difficulty_analysis.groupby('Difficulty Group')['Correct Rate'].apply(list).tolist()
x_labels = [str(group) for group in difficulty_analysis['Difficulty Group'].cat.categories]

# 创建箱线图
boxplot = Boxplot()
boxplot.add_xaxis(x_labels)
boxplot.add_yaxis("正确率", boxplot.prepare_data(box_data))
boxplot.set_global_opts(
    title_opts=opts.TitleOpts(title="按题目难度分组的正确率箱线图"),
    xaxis_opts=opts.AxisOpts(name="难度分组"),
    yaxis_opts=opts.AxisOpts(name="正确率")
)

# 渲染图表
boxplot.render("difficulty_correct_rate_boxplot.html")

from pyecharts.charts import Scatter
from pyecharts import options as opts

# 提取数据
x_data = difficulty_analysis['Average Score'].tolist()
y_data = difficulty_analysis['Correct Rate'].tolist()
titles = difficulty_analysis['title_ID'].tolist()  # 将 title_ID 作为标签

# 设置高难度低正确率的阈值（可调整）
difficulty_threshold = 1.0  # 难度阈值
correct_rate_threshold = 0.2  # 正确率阈值

# 根据阈值筛选出异常题目
anomalous_titles = [title for score, rate, title in zip(x_data, y_data, titles)
                    if score > difficulty_threshold and rate < correct_rate_threshold]

# 创建散点图
scatter = (
    Scatter()
    .add_xaxis(x_data)
    .add_yaxis(
        "题目正确率",
        [list(z) for z in zip(y_data, titles)],  # 将 title_ID 作为标签
        label_opts=opts.LabelOpts(is_show=True, formatter="{@[1]}")  # 显示 title_ID 作为标签
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="题目难度与正确率关系（散点图）"),
        xaxis_opts=opts.AxisOpts(name="题目难度"),
        yaxis_opts=opts.AxisOpts(name="正确率")
    )
)

# 渲染图表
scatter.render("difficulty_correct_rate_scatter.html")


C:\Windows\Temp\ipykernel_5872\2286688004.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  difficulty_analysis = merged_data.groupby('title_ID').apply(
C:\Windows\Temp\ipykernel_5872\2286688004.py:18: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  box_data = difficulty_analysis.groupby('Difficulty Group')['Correct Rate'].apply(list).tolist()


'D:\\Python\\数据可视化\\实验六\\difficulty_correct_rate_scatter.html'

In [ ]:
# 加载数据集
submit_records = pd.read_csv('./数据/Data_SubmitRecord/SubmitRecord-Class1.csv')
question_attributes = pd.read_csv('./数据/题目属性.csv')
student_attributes = pd.read_csv('./数据/学生属性.csv')

In [41]:
# 导入必要的库
import pandas as pd
from pyecharts.charts import Bar, Scatter, Line, Page
from pyecharts import options as opts

# 加载数据集
submit_records = pd.read_csv('./数据/Data_SubmitRecord/SubmitRecord-Class1.csv')
question_attributes = pd.read_csv('./数据/题目属性.csv')
student_attributes = pd.read_csv('./数据/学生属性.csv')

# 合并数据集
merged_data = pd.merge(submit_records, question_attributes, on='title_ID', how='left')
merged_data = pd.merge(merged_data, student_attributes, on='student_ID', how='left')

# 保存合并后的数据到 Excel 文件
merged_data.to_excel('./数据/合并后的数据.xlsx', index=False)
print("合并后的数据已保存为 '合并后的数据.xlsx' 文件。")


合并后的数据已保存为 '合并后的数据.xlsx' 文件。


In [54]:
# 导入必要的库
import pandas as pd
from pyecharts.charts import Bar, Scatter, Line, Page
from pyecharts import options as opts

# 加载和合并数据集
submit_records = pd.read_csv('./数据/Data_SubmitRecord/SubmitRecord-Class1.csv')
question_attributes = pd.read_csv('./数据/题目属性.csv')
student_attributes = pd.read_csv('./数据/学生属性.csv')

# 合并数据
merged_data = pd.merge(submit_records, question_attributes, on='title_ID', how='left')
merged_data = pd.merge(merged_data, student_attributes, on='student_ID', how='left')

# 保存合并后的数据到 Excel 文件
merged_data.to_excel('./数据/合并后的数据.xlsx', index=False)

# 1. 知识点掌握程度分析
knowledge_mastery = merged_data.groupby('knowledge')['score_x'].mean().reset_index()
knowledge_mastery.columns = ['知识点', '平均得分']

bar_knowledge = (
    Bar()
    .add_xaxis(knowledge_mastery['知识点'].tolist())
    .add_yaxis("平均得分", knowledge_mastery['平均得分'].tolist())
    .set_global_opts(
        title_opts=opts.TitleOpts(title="知识点掌握程度分析"),
        xaxis_opts=opts.AxisOpts(name="知识点"),
        yaxis_opts=opts.AxisOpts(name="平均得分")
    )
)

# 2. 学习行为模式分析 - 答题高峰时段
merged_data['time'] = pd.to_datetime(merged_data['time'], unit='s')
merged_data['hour'] = merged_data['time'].dt.hour
hourly_distribution = merged_data.groupby('hour').size().reset_index(name='答题次数')

bar_time = (
    Bar()
    .add_xaxis([f"{i}点" for i in hourly_distribution['hour']])
    .add_yaxis("答题次数", hourly_distribution['答题次数'].tolist())
    .set_global_opts(
        title_opts=opts.TitleOpts(title="答题高峰时段分布"),
        xaxis_opts=opts.AxisOpts(name="时段"),
        yaxis_opts=opts.AxisOpts(name="答题次数")
    )
)

# 3. 偏好题型分析
question_type_distribution = merged_data['knowledge'].value_counts().reset_index()
question_type_distribution.columns = ['题型', '答题次数']

bar_type = (
    Bar()
    .add_xaxis(question_type_distribution['题型'].tolist())
    .add_yaxis("答题次数", question_type_distribution['答题次数'].tolist())
    .set_global_opts(
        title_opts=opts.TitleOpts(title="题型偏好分析"),
        xaxis_opts=opts.AxisOpts(name="题型"),
        yaxis_opts=opts.AxisOpts(name="答题次数")
    )
)

# 4. 正确答题率分析
# 正确答题率分析，避免警告
correct_rate = merged_data.groupby('knowledge')['state'].apply(
    lambda x: (x == 'Absolutely_Correct').mean()
).reset_index()
correct_rate.columns = ['题型', '正确答题率']

line_correct_rate = (
    Line()
    .add_xaxis(correct_rate['题型'].tolist())
    .add_yaxis("正确答题率", (correct_rate['正确答题率'] * 100).tolist(), label_opts=opts.LabelOpts(formatter="{c}%"))
    .set_global_opts(
        title_opts=opts.TitleOpts(title="正确答题率分析"),
        xaxis_opts=opts.AxisOpts(name="题型"),
        yaxis_opts=opts.AxisOpts(name="正确答题率 (%)")
    )
)



# 5. 学习模式与知识掌握关系 - 答题耗时分析
learning_behavior = merged_data.groupby('timeconsume')['score_x'].mean().reset_index()
learning_behavior.columns = ['答题耗时', '平均得分']

line_timeconsume = (
    Line()
    .add_xaxis(learning_behavior['答题耗时'].tolist())
    .add_yaxis("平均得分", learning_behavior['平均得分'].tolist(), is_smooth=True)
    .set_global_opts(
        title_opts=opts.TitleOpts(title="答题耗时与知识掌握程度关系"),
        xaxis_opts=opts.AxisOpts(name="答题耗时 (秒)"),
        yaxis_opts=opts.AxisOpts(name="平均得分")
    )
)

# 6. 每道题的得分与答题量情况（气泡图）
# 计算每个题目的平均得分和答题量
title_score_data = merged_data.groupby('title_ID').agg(
    avg_score=('score_x', 'mean'),
    count=('score_x', 'size')
).reset_index()

title_list = title_score_data['title_ID'].tolist()
avg_score_list = title_score_data['avg_score'].tolist()
count_list = title_score_data['count'].tolist()

scatter = (
    Scatter()
    .add_xaxis(title_list)
    .add_yaxis(
        "Average Score",
        avg_score_list,
        symbol_size=[min(50, max(10, c / 2)) for c in count_list],  # 控制气泡大小范围
        itemstyle_opts=opts.ItemStyleOpts(opacity=0.7)  # 设置气泡透明度
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="每道题的得分与答题量关系"),
        xaxis_opts=opts.AxisOpts(name="题目ID", axislabel_opts={"rotate": 45}),
        yaxis_opts=opts.AxisOpts(name="平均得分", min_=0, max_=4),  # 假设得分满分为4
        tooltip_opts=opts.TooltipOpts(trigger="item", formatter="{b}: 平均得分 {c}"),
        legend_opts=opts.LegendOpts(pos_top="5%", pos_right="5%"),  # 设置图例位置为右上角
        visualmap_opts=opts.VisualMapOpts(
            type_="size",
            max_=50,
            min_=10,
            dimension=1,
            pos_right="5%",  # 右对齐
            pos_top="15%",   # 图例下方
        )
    )
)

# 生成HTML看板
page = Page(layout=Page.DraggablePageLayout)
page.add(bar_knowledge, bar_time, bar_type, line_correct_rate, line_timeconsume, scatter)

# 渲染为 HTML 文件
page.render("learning_analysis_dashboard.html")


'D:\\Python\\数据可视化\\实验六\\learning_analysis_dashboard.html'

In [5]:
# 导入必要的库
import pandas as pd
from pyecharts.charts import Bar, Line, Pie, Scatter, Page
from pyecharts import options as opts

# 加载和合并数据集
submit_records = pd.read_csv('./数据/Data_SubmitRecord/SubmitRecord-Class1.csv')
question_attributes = pd.read_csv('./数据/题目属性.csv')
student_attributes = pd.read_csv('./数据/学生属性.csv')

# 合并数据
merged_data = pd.merge(submit_records, question_attributes, on='title_ID', how='left')
merged_data = pd.merge(merged_data, student_attributes, on='student_ID', how='left')

# 加载数据
file_path = './数据/合并后的数据.xlsx'
data = pd.read_excel(file_path)

# 确保 timeconsume 列为数值，无法转换的设置为 NaN
data['timeconsume'] = pd.to_numeric(data['timeconsume'], errors='coerce')

# 1. 编程语言使用偏好分析
language_distribution = data['method'].value_counts().reset_index()
language_distribution.columns = ['编程语言', '答题次数']

pie_language = (
    Pie()
    .add(
        "编程语言",
        [list(z) for z in zip(language_distribution['编程语言'], language_distribution['答题次数'])]
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="编程语言使用偏好"),
        legend_opts=opts.LegendOpts(orient="vertical", pos_top="15%", pos_left="2%")
    )
    .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c}次"))
)

# 2. 编程语言与平均得分分析
language_score = data.groupby('method')['score_x'].mean().reset_index()
language_score.columns = ['编程语言', '平均得分']

bar_language_score = (
    Bar()
    .add_xaxis(language_score['编程语言'].tolist())
    .add_yaxis("平均得分", language_score['平均得分'].tolist())
    .set_global_opts(
        title_opts=opts.TitleOpts(title="编程语言与平均得分"),
        xaxis_opts=opts.AxisOpts(name="编程语言"),
        yaxis_opts=opts.AxisOpts(name="平均得分", min_=0, max_=4)
    )
)

# 3. 编程语言与答题耗时分析
language_timeconsume = data.dropna(subset=['timeconsume']).groupby('method')['timeconsume'].mean().reset_index()
language_timeconsume.columns = ['编程语言', '平均答题耗时']

line_language_timeconsume = (
    Line()
    .add_xaxis(language_timeconsume['编程语言'].tolist())
    .add_yaxis("平均耗时 (秒)", language_timeconsume['平均答题耗时'].tolist(), label_opts=opts.LabelOpts(formatter="{c} s"))
    .set_global_opts(
        title_opts=opts.TitleOpts(title="编程语言与答题耗时"),
        xaxis_opts=opts.AxisOpts(name="编程语言"),
        yaxis_opts=opts.AxisOpts(name="平均耗时 (秒)")
    )
)

# 4. 知识点掌握情况分析
knowledge_mastery = data.groupby('knowledge')['score_x'].mean().reset_index()
knowledge_mastery.columns = ['知识点', '平均得分']

bar_knowledge = (
    Bar()
    .add_xaxis(knowledge_mastery['知识点'].tolist())
    .add_yaxis("平均得分", knowledge_mastery['平均得分'].tolist())
    .set_global_opts(
        title_opts=opts.TitleOpts(title="知识点掌握情况"),
        xaxis_opts=opts.AxisOpts(name="知识点"),
        yaxis_opts=opts.AxisOpts(name="平均得分")
    )
)

# 5. 答题活跃时段分析
data['time'] = pd.to_datetime(data['time'], unit='s')
data['hour'] = data['time'].dt.hour
hourly_distribution = data.groupby('hour').size().reset_index(name='答题次数')

bar_time = (
    Bar()
    .add_xaxis([f"{i}点" for i in hourly_distribution['hour']])
    .add_yaxis("答题次数", hourly_distribution['答题次数'].tolist())
    .set_global_opts(
        title_opts=opts.TitleOpts(title="答题活跃时段分布"),
        xaxis_opts=opts.AxisOpts(name="时段"),
        yaxis_opts=opts.AxisOpts(name="答题次数")
    )
)

# 6. 偏好题型分析
question_type_distribution = data['knowledge'].value_counts().reset_index()
question_type_distribution.columns = ['题型', '答题次数']

bar_type = (
    Bar()
    .add_xaxis(question_type_distribution['题型'].tolist())
    .add_yaxis("答题次数", question_type_distribution['答题次数'].tolist())
    .set_global_opts(
        title_opts=opts.TitleOpts(title="题型偏好分析"),
        xaxis_opts=opts.AxisOpts(name="题型"),
        yaxis_opts=opts.AxisOpts(name="答题次数")
    )
)

# 7. 正确答题率分析
correct_rate = data.groupby('knowledge')['state'].apply(
    lambda x: (x == 'Absolutely_Correct').mean()
).reset_index()
correct_rate.columns = ['题型', '正确答题率']

line_correct_rate = (
    Line()
    .add_xaxis(correct_rate['题型'].tolist())
    .add_yaxis("正确答题率", (correct_rate['正确答题率'] * 100).tolist(), label_opts=opts.LabelOpts(formatter="{c}%"))
    .set_global_opts(
        title_opts=opts.TitleOpts(title="正确答题率分析"),
        xaxis_opts=opts.AxisOpts(name="题型"),
        yaxis_opts=opts.AxisOpts(name="正确答题率 (%)")
    )
)

# 8. 学习模式与知识掌握关系 - 答题耗时分析
learning_behavior = data.groupby('timeconsume')['score_x'].mean().reset_index()
learning_behavior.columns = ['答题耗时', '平均得分']

line_timeconsume = (
    Line()
    .add_xaxis(learning_behavior['答题耗时'].tolist())
    .add_yaxis("平均得分", learning_behavior['平均得分'].tolist(), is_smooth=True)
    .set_global_opts(
        title_opts=opts.TitleOpts(title="答题耗时与知识掌握程度关系"),
        xaxis_opts=opts.AxisOpts(name="答题耗时 (秒)"),
        yaxis_opts=opts.AxisOpts(name="平均得分")
    )
)

# 9. 每道题的得分与答题量情况（气泡图）
title_score_data = data.groupby('title_ID').agg(
    avg_score=('score_x', 'mean'),
    count=('score_x', 'size')
).reset_index()

title_list = title_score_data['title_ID'].tolist()
avg_score_list = title_score_data['avg_score'].tolist()
count_list = title_score_data['count'].tolist()

scatter = (
    Scatter()
    .add_xaxis(title_list)
    .add_yaxis(
        "Average Score",
        avg_score_list,
        symbol_size=[min(50, max(10, c / 2)) for c in count_list],  # 控制气泡大小范围
        itemstyle_opts=opts.ItemStyleOpts(opacity=0.7)  # 设置气泡透明度
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="每道题的得分与答题量关系"),
        xaxis_opts=opts.AxisOpts(name="题目ID", axislabel_opts={"rotate": 45}),
        yaxis_opts=opts.AxisOpts(name="平均得分", min_=0, max_=4),  # 假设得分满分为4
        tooltip_opts=opts.TooltipOpts(trigger="item", formatter="{b}: 平均得分 {c}"),
        legend_opts=opts.LegendOpts(pos_top="5%", pos_right="5%"),  # 设置图例位置为右上角
        visualmap_opts=opts.VisualMapOpts(
            type_="size",
            max_=50,
            min_=10,
            dimension=1,
            pos_right="5%",  # 右对齐
            pos_top="15%",   # 图例下方
        )
    )
)

# 10. 年龄与答题表现分析
age_score = data.groupby('age')['score_x'].mean().reset_index()
age_score.columns = ['年龄', '平均得分']

bar_age_score = (
    Bar()
    .add_xaxis(age_score['年龄'].astype(str).tolist())
    .add_yaxis("平均得分", age_score['平均得分'].tolist())
    .set_global_opts(
        title_opts=opts.TitleOpts(title="年龄与答题表现"),
        xaxis_opts=opts.AxisOpts(name="年龄"),
        yaxis_opts=opts.AxisOpts(name="平均得分")
    )
)

# 生成包含所有图表的HTML看板
page = Page(layout=Page.DraggablePageLayout)
page.add(
    pie_language, 
    bar_language_score, 
    line_language_timeconsume, 
    bar_knowledge, 
    bar_time, 
    bar_type, 
    line_correct_rate, 
    line_timeconsume, 
    scatter, 
    bar_age_score
)

# 渲染为 HTML 文件
page.render("learning_analysis_full_dashboard.html")


'D:\\Python\\数据可视化\\实验六\\learning_analysis_full_dashboard.html'

In [62]:
# 导入必要的库
import pandas as pd
import glob
import os

# 加载题目和学生的属性数据，只需加载一次
question_attributes = pd.read_csv('./数据/题目属性.csv')
student_attributes = pd.read_csv('./数据/学生属性.csv')

# 获取所有班级数据文件的路径
file_paths = glob.glob('./数据/Data_SubmitRecord/*.csv')

# 遍历每个班级的数据文件，合并并保存为 Excel 文件
for file_path in file_paths:
    # 读取班级提交记录数据
    submit_records = pd.read_csv(file_path)
    
    # 提取班级名称（假设文件名包含班级信息）
    class_name = os.path.splitext(os.path.basename(file_path))[0]
    
    # 合并数据
    merged_data = pd.merge(submit_records, question_attributes, on='title_ID', how='left')
    merged_data = pd.merge(merged_data, student_attributes, on='student_ID', how='left')
    
    # 保存合并后的数据到 Excel 文件
    output_path = f'./数据/{class_name}_merged_data.xlsx'
    merged_data.to_excel(output_path, index=False)
    print(f"合并后的数据已保存为 '{output_path}' 文件。")


Exception ignored in: <function ZipFile.__del__ at 0x000001E4EDF15D30>
Traceback (most recent call last):
  File "D:\Anaconda3\envs\shumo\lib\zipfile.py", line 1828, in __del__
    self.close()
  File "D:\Anaconda3\envs\shumo\lib\zipfile.py", line 1845, in close
    self.fp.seek(self.start_dir)
ValueError: seek of closed file


合并后的数据已保存为 './数据/SubmitRecord-Class1_merged_data.xlsx' 文件。
合并后的数据已保存为 './数据/SubmitRecord-Class10_merged_data.xlsx' 文件。
合并后的数据已保存为 './数据/SubmitRecord-Class11_merged_data.xlsx' 文件。
合并后的数据已保存为 './数据/SubmitRecord-Class12_merged_data.xlsx' 文件。
合并后的数据已保存为 './数据/SubmitRecord-Class13_merged_data.xlsx' 文件。
合并后的数据已保存为 './数据/SubmitRecord-Class14_merged_data.xlsx' 文件。
合并后的数据已保存为 './数据/SubmitRecord-Class15_merged_data.xlsx' 文件。
合并后的数据已保存为 './数据/SubmitRecord-Class2_merged_data.xlsx' 文件。
合并后的数据已保存为 './数据/SubmitRecord-Class3_merged_data.xlsx' 文件。
合并后的数据已保存为 './数据/SubmitRecord-Class4_merged_data.xlsx' 文件。
合并后的数据已保存为 './数据/SubmitRecord-Class5_merged_data.xlsx' 文件。
合并后的数据已保存为 './数据/SubmitRecord-Class6_merged_data.xlsx' 文件。
合并后的数据已保存为 './数据/SubmitRecord-Class7_merged_data.xlsx' 文件。
合并后的数据已保存为 './数据/SubmitRecord-Class8_merged_data.xlsx' 文件。
合并后的数据已保存为 './数据/SubmitRecord-Class9_merged_data.xlsx' 文件。


In [61]:
import pandas as pd
from pyecharts.charts import Bar, Line, Pie, Scatter, Page
from pyecharts import options as opts
import os

# 设置数据路径，假设所有班级的 CSV 数据文件存放在 './数据/Data_SubmitRecord/' 目录下
file_paths = [
    './数据/Data_SubmitRecord/SubmitRecord-Class1.csv',
    './数据/Data_SubmitRecord/SubmitRecord-Class2.csv',
    './数据/Data_SubmitRecord/SubmitRecord-Class3.csv',
    # 添加其他班级数据文件路径
]

# 合并和清洗每个班级的数据，并分别保存为 Excel 和 HTML 看板
for file_path in file_paths:
    # 读取班级数据
    class_data = pd.read_csv(file_path)
    
    # 提取班级名称（假设文件名包含班级信息）
    class_name = os.path.splitext(os.path.basename(file_path))[0]
    class_data['class_name'] = class_name  # 添加班级列

    # 检查是否包含所需的列
    required_columns = ['method', 'score_x', 'timeconsume', 'time', 'knowledge']
    missing_columns = [col for col in required_columns if col not in class_data.columns]
    
    if missing_columns:
        print(f"{class_name} 缺少列: {missing_columns}")
        continue  # 如果缺少列，跳过该文件的处理

    # 数据清洗
    class_data['timeconsume'] = pd.to_numeric(class_data['timeconsume'], errors='coerce')
    class_data['time'] = pd.to_datetime(class_data['time'], unit='s')
    
    # 保存每个班级的清洗合并数据为 Excel 文件
    class_data.to_excel(f'./数据/{class_name}_merged_data.xlsx', index=False)

    # 1. 编程语言使用偏好分析
    language_distribution = class_data['method'].value_counts().reset_index()
    language_distribution.columns = ['编程语言', '答题次数']
    pie_language = (
        Pie()
        .add(
            "编程语言",
            [list(z) for z in zip(language_distribution['编程语言'], language_distribution['答题次数'])]
        )
        .set_global_opts(
            title_opts=opts.TitleOpts(title="编程语言使用偏好"),
            legend_opts=opts.LegendOpts(orient="vertical", pos_top="15%", pos_left="2%")
        )
        .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c}次"))
    )

    # 2. 编程语言与平均得分分析
    language_score = class_data.groupby('method')['score_x'].mean().reset_index()
    language_score.columns = ['编程语言', '平均得分']
    bar_language_score = (
        Bar()
        .add_xaxis(language_score['编程语言'].tolist())
        .add_yaxis("平均得分", language_score['平均得分'].tolist())
        .set_global_opts(
            title_opts=opts.TitleOpts(title="编程语言与平均得分"),
            xaxis_opts=opts.AxisOpts(name="编程语言"),
            yaxis_opts=opts.AxisOpts(name="平均得分", min_=0, max_=4)
        )
    )

    # 3. 编程语言与答题耗时分析
    language_timeconsume = class_data.dropna(subset=['timeconsume']).groupby('method')['timeconsume'].mean().reset_index()
    language_timeconsume.columns = ['编程语言', '平均答题耗时']
    line_language_timeconsume = (
        Line()
        .add_xaxis(language_timeconsume['编程语言'].tolist())
        .add_yaxis("平均耗时 (秒)", language_timeconsume['平均答题耗时'].tolist(), label_opts=opts.LabelOpts(formatter="{c} s"))
        .set_global_opts(
            title_opts=opts.TitleOpts(title="编程语言与答题耗时"),
            xaxis_opts=opts.AxisOpts(name="编程语言"),
            yaxis_opts=opts.AxisOpts(name="平均耗时 (秒)")
        )
    )

    # 4. 知识点掌握情况分析
    knowledge_mastery = class_data.groupby('knowledge')['score_x'].mean().reset_index()
    knowledge_mastery.columns = ['知识点', '平均得分']
    bar_knowledge = (
        Bar()
        .add_xaxis(knowledge_mastery['知识点'].tolist())
        .add_yaxis("平均得分", knowledge_mastery['平均得分'].tolist())
        .set_global_opts(
            title_opts=opts.TitleOpts(title="知识点掌握情况"),
            xaxis_opts=opts.AxisOpts(name="知识点"),
            yaxis_opts=opts.AxisOpts(name="平均得分")
        )
    )

    # 创建每个班级的页面
    page = Page(layout=Page.DraggablePageLayout)
    page.add(pie_language, bar_language_score, line_language_timeconsume, bar_knowledge)
    
    # 渲染当前班级的页面
    page_file_path = f"./数据/{class_name}_dashboard.html"
    page.render(page_file_path)

# 生成包含所有班级链接的主页面
with open("all_classes_dashboard.html", "w", encoding="utf-8") as f:
    f.write("<html><head><title>所有班级数据看板</title></head><body>")
    f.write("<h1>所有班级数据看板</h1><ul>")
    for file_path in file_paths:
        class_name = os.path.splitext(os.path.basename(file_path))[0]
        f.write(f'<li><a href="./数据/{class_name}_dashboard.html">{class_name} 的数据看板</a></li>')
    f.write("</ul></body></html>")


SubmitRecord-Class1 缺少列: ['score_x', 'knowledge']
SubmitRecord-Class2 缺少列: ['score_x', 'knowledge']
SubmitRecord-Class3 缺少列: ['score_x', 'knowledge']
